# **SOLID Principles**

https://arjancodes.com/blog/solid-principles-in-python-programming/

https://youtu.be/pTB30aXS77U

**¿Cuáles son los principios SÓLIDOS?**

1. Principio de Responsabilidad Única (SRP)

El Principio de Responsabilidad Única aboga por que una clase o módulo tenga una sola razón para cambiar. En términos más simples, debería hacer una cosa y hacerlo bien. Al adherirse a SRP, su código se vuelve más modular, lo que facilita su comprensión y mantenimiento.

2. Principio abierto-cerrado (OCP)

El principio abierto-cerrado establece que las entidades de software deben estar abiertas a la extensión pero cerradas a la modificación. Esto significa que debería poder ampliar el comportamiento de una clase sin modificarla.

3. Principio de sustitución de Liskov (LSP)

El principio de sustitución de Liskov establece que los objetos de un programa deben ser reemplazables con instancias de sus subtipos sin alterar la corrección del programa. En otras palabras, una subclase debería poder reemplazar su clase principal sin romper el código.

4. Principio de segregación de interfaz (ISP)

El principio de segregación de interfaces establece que no se debe obligar a los clientes a depender de métodos que no utilizan. Esto significa que no debería tener que implementar métodos que no necesita.

5. Principio de inversión de dependencia (DIP)

El principio de inversión de dependencia establece que los módulos de alto nivel no deberían depender de los módulos de bajo nivel, sino que ambos deberían depender de abstracciones. Esto significa que no debería tener que cambiar su código cuando cambie la implementación de un módulo.

**Principio de Responsabilidad Única (SRP)**

In [1]:
class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)

    def total_price(self):
        total = 0
        for quantity, price in zip(self.quantities, self.prices):
            total += quantity * price
        return total

    def pay(self, payment_type: str, security_code):
        if payment_type == "debit":
            print("Processing debit payment type")
            print(f"Verifying security code: {security_code}")
            self.status = "paid"
        elif payment_type == "credit":
            print("Processing credit payment type")
            print(f"Verifying security code: {security_code}")
            self.status = "paid"
        else:
            raise Exception(f"Unknown payment type: {payment_type}")

In [ ]:
order = Order()
order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)
print(order.total_price())
order.pay("debit", "0372846")

Este código viola el SRP porque es responsable de gestionar el pedido y el pago. Esto hace que nuestro código esté altamente acoplado y hace que sea más difícil de entender, mantener y probar.

Refactoricemos este código para cumplir con el SRP.

In [ ]:
class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)


class PaymentProcessor:
    def pay(self, order: Order, security_code: str):
        print("Processing payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"

In [ ]:
order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = PaymentProcessor()
processor.pay(order, "0372846")
print(order.status)

Al separar nuestras responsabilidades, podemos agregar nuevos métodos de pago con facilidad sin tener que modificar la clase de Pedido.

Nota: Este código aún viola el SRP porque el pedido es responsable tanto de los precios como de las cantidades y podría mejorarse separando estas preocupaciones.

**Principio abierto-cerrado (OCP)**

Podemos mejorar aún más este código si seguimos el principio abierto-cerrado.

Si deseamos agregar un nuevo método de pago, tendríamos que realizar modificaciones en la clase PaymentProcessor. Esto viola el principio abierto-cerrado que, como sabemos, establece que las entidades de software deben estar abiertas a la extensión pero cerradas a la modificación.

Modifiquemos este código para cumplir con el OCP.

In [2]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)


class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order, security_code: str):
        ...


class CreditCardPaymentProcessor(PaymentProcessor):
    def pay(self, order, security_code: str):
        print("Processing credit card payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


class DebitCardPaymentProcessor(PaymentProcessor):
    def pay(self, order, security_code: str):
        print("Processing debit card payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):
    def pay(self, order, security_code: str):
        print("Processing paypal payment")
        print(f"Verifying email: {security_code}")
        order.status = "paid"

In [3]:
order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = CreditCardPaymentProcessor()
processor.pay(order, "0372846")
print(order.status)

Processing credit card payment
Verifying security code: 0372846
paid


Ahora que el código se adhiere al OCP, está cerrado para modificación y abierto para extensión porque podemos agregar nuevos métodos de pago sin modificar la clase PaymentProcessor.

**Principio de sustitución de Liskov (LSP)**

En nuestra refactorización, hemos introducido una nueva violación de los principios SOLID. Paypal utiliza direcciones de correo electrónico para la verificación, mientras que las tarjetas de crédito y débito utilizan códigos de seguridad. **Esto significa que estamos abusando del principio de sustitución de Liskov porque estamos usando una subclase de una manera que no es compatible con su clase principal.** Esto se debe al concepto de Diseño por contrato, que en este contexto dicta que las clases deben cumplir con el "contrato" establecido por su interfaz para lograr coherencia e integridad.

Refactoricemos este código para cumplir con el LSP.

In [ ]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)


class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order):
        ...


class DebitPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code

    def pay(self, order):
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):
    def __init__(self, email_address: str):
        self.email_address = email_address

    def pay(self, order):
        print("Processing paypal payment")
        print(f"Verifying email: {self.email_address}")
        order.status = "paid"

In [ ]:
order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = PaypalPaymentProcessor("hi@arjancodes.com")
processor.pay(order)
print(order.status)

Ahora el código se adhiere al LSP porque estamos usando las subclases de una manera que es compatible con su clase principal.

**Principio de segregación de interfaz (ISP)**

Este principio establece que no se debe obligar a los clientes a depender de métodos que no utilizan. Esto significa que es mejor tener interfaces que se adapten a tareas específicas en lugar de una interfaz de propósito general.

Te daré un ejemplo donde agregamos la posibilidad de enviar al usuario un SMS para autenticar su pago.

In [4]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)


class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order):
        ...

    @abstractmethod
    def auth_sms(self, order, code: str):
        ...


class DebitPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code
        self.authenticated = False

    def pay(self, order):
        if not self.authenticated:
            raise Exception("Not authenticated")
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"

    def auth_sms(self, order, code: str):
        print("Authenticating via SMS")
        self.authenticated = True


class CreditPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"

    def auth_sms(self, order, code: str):
        raise Exception("Not implemented")


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = DebitPaymentProcessor("0372846")
processor.auth_sms(order, "12345")
processor.pay(order)
print(order.status)


Authenticating via SMS
Processing debit payment type
Verifying security code: 0372846
paid


Este código viola el ISP porque la clase CreditPaymentProcessor se ve obligada a implementar el método auth_sms, aunque no lo utilice. Esto no solo significa que terminaremos escribiendo más código, sino que podría causar errores si nos olvidamos de implementar el método.

Refactoricemos este código para cumplir con el ISP.

In [ ]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)


class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order: Order):
        ...


class SmsPaymentProcessor(PaymentProcessor):
    @abstractmethod
    def auth_sms(self, order: Order, code: str):
        ...


class DebitPaymentProcessor(SmsPaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code
        self.authenticated = False

    def pay(self, order):
        if not self.authenticated:
            raise Exception("Not authenticated")
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"

    def auth_sms(self, order, code: str):
        print("Authenticating via SMS")
        self.authenticated = True


class CreditPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = DebitPaymentProcessor("0372846")
processor.auth_sms(order, "12345")
processor.pay(order)

print(order.status)


Ahora, podríamos mejorar aún más este código separando la lógica de autorización del procesador de pagos.

In [ ]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)


class SMSAuthorizer:
    def __init__(self):
        self.authenticated = False

    def verify_code(self, code: str):
        print("Verifying code")
        self.authenticated = True

    def is_authenticated(self):
        return self.authenticated


class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order: Order):
        ...


class DebitPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str, authorizer: SMSAuthorizer):
        self.security_code = security_code
        self.authorizer = authorizer

    def pay(self, order):
        if not self.authorizer.is_authenticated():
            raise Exception("Not authenticated")
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code

    def pay(self, order: Order):
        print("Processing credit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

authorizer = SMSAuthorizer()
authorizer.verify_code("12345")

processor = DebitPaymentProcessor("0372846", authorizer)

processor.pay(order)

print(order.status)


**Principio de inversión de dependencia (DIP)**

Podríamos mejorar aún más este código si cumpliéramos el principio de inversión de dependencia. El principio de inversión de dependencia establece que los módulos de alto nivel no deberían depender de los módulos de bajo nivel, sino que ambos deberían depender de abstracciones. Esto significa que no debería tener que cambiar otras secciones de su código cuando cambia la implementación de una clase.

En la práctica, esto significa que nuestro procesador de pagos no debería preocuparse por cómo se valida su pago, ya sea mediante SMS, un cheque robot o un correo electrónico.

Refactoricemos este código para cumplir con el DIP.

In [ ]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)


class Authorizer(ABC):
    @abstractmethod
    def is_authenticated(self):
        ...


class SMSAuthorizer(Authorizer):
    def __init__(self):
        self.authenticated = False

    def verify_code(self, code: str):
        print("Verifying code")
        self.authenticated = True

    def is_authenticated(self):
        return self.authenticated


class NotARobotAuthorizer(Authorizer):
    def __init__(self):
        self.authenticated = False

    def ask(self):
        print("Are you a robot?!!! [┐∵]┘")
        self.authenticated = True

    def is_authenticated(self):
        return self.authenticated


class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order: Order):
        ...


class DebitPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str, authorizer: Authorizer):
        self.security_code = security_code
        self.authorizer = authorizer

    def pay(self, order: Order):
        if not self.authorizer.is_authenticated():
            raise Exception("Not authenticated")
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code: str):
        self.security_code = security_code

    def pay(self, order: Order):
        print("Processing credit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

authorizer = NotARobotAuthorizer()

authorizer.ask()

processor = DebitPaymentProcessor("0372846", authorizer)

processor.pay(order)

print(order.status)


Ahora el código se adhiere al DIP porque el módulo de alto nivel DebitPaymentProcessor no depende de la clase de bajo nivel sino que depende del Autorizador de abstracción de alto nivel. Esto significa que podemos cambiar la implementación de la clase Authorizer que usamos sin tener que cambiar la clase DebitPaymentProcessor.


Conclusiones

Como puede ver, al aplicar cuidadosamente los principios de diseño SOLID, puede mejorar la capacidad de mantenimiento y la escalabilidad de sus aplicaciones. La aplicación práctica de estos principios puede conducir a un software más cohesivo, menos acoplado y más fácil de mejorar a largo plazo.